## Testing hypothesis 3
---

**_Hypothesis_**: There are some publisher getting higher average review/score.

   - **Metric**: Compare the mean review/score of the books published by the same publisher.

   - **Model**: Linear Regression.

   - **Description**:
   
     - Add a column for each publisher's average review/score.
     - Use the average review/score as the predictor variable and the helpfulness rating as the target variable.
     - Train a linear regression model to predict helpfulness ratings based on the average review/score.

**Missing Values**:
  
- `publisher`: set missing values as empty string
- `review/score`: remove the entire sample

**Data Transformation**:

- `publisher`: GroupBy publisher.
- `review/score`: Compute the average review/score for each publisher.

---

In [ ]:
# Connect to MongoDB

import pymongo

client = pymongo.MongoClient('mongodb://localhost:27017/')
database = client['spark_db']
books = database['books_hypothesis_7']

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Group by publisher and calculate average rating
pipeline_average_rating = {'$group':{
                                '_id':'$publisher',
                                'average_rating':{'$avg':'review/score'}
                                }
                           }

